In [ ]:
# Common part ----------------
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.utilities import ArxivAPIWrapper

from dotenv import load_dotenv
load_dotenv(override=True)

llm = ChatGroq(temperature=0, model_name= "qwen/qwen3-32b")

In [ ]:
def parse_wiki_info(text: str) -> dict:
    """fomat of the wiki search result"""
    lines = text.strip().split('\n')
    result = {}
    current_key = None
    summary_lines = []

    for line in lines:
        if line.startswith('Page:'):
            result['Page'] = line[len('Page:'):].strip()
            current_key = None
        elif line.startswith('Summary:'):
            current_key = 'Summary'
            summary_lines.append(line[len('Summary:'):].strip())
        elif current_key == 'Summary':
            summary_lines.append(line.strip())

    if summary_lines:
        result['Summary'] = ' '.join(summary_lines)

    return result

def parse_arxiv_info(text: str) -> dict:
    """fomat of the arxiv search result"""
    lines = text.strip().split('\n')
    result = {}
    current_key = None
    summary_lines = []

    for line in lines:
        if line.startswith('Published:'):
            result['Published'] = line[len('Published:'):].strip()
            current_key = None
        elif line.startswith('Title:'):
            result['Title'] = line[len('Title:'):].strip()
            current_key = None
        elif line.startswith('Authors:'):
            result['Authors'] = line[len('Authors:'):].strip()
            current_key = None
        elif line.startswith('Summary:'):
            current_key = 'Summary'
            summary_lines.append(line[len('Summary:'):].strip())
        elif current_key == 'Summary':
            summary_lines.append(line.strip())

    if summary_lines:
        result['Summary'] = ' '.join(summary_lines)

    return result

In [29]:
# Step 0: Define tools and model
from langchain_core.tools import tool
from pprint import pprint

@tool(infer_schema=True, parse_docstring=True)
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

@tool(infer_schema=True, parse_docstring=True)
def web_search(query: str) -> str:
    """Searches the web for the query and returns the results.
    
    Args:
        query: The query to search for.
    """
    web_search_tool = TavilySearch(
                max_results=3,
                include_answer=False,
                include_raw_content=False,
                include_images=False,
            )
    web_results = web_search_tool.invoke({'query': query})
    return str(web_results)

@tool(infer_schema=True, parse_docstring=True)
def wiki_search(query: str) -> str:
    """Searches Wikipedia for the query and returns the results.
    
    Args:
        query: The query to search for.
    """ 
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=3, lang="ko"))
    wiki_results = wikipedia.run(query)
    wiki_results = wiki_results.split("\n\n")
    wiki_results = [wiki_result for wiki_result in wiki_results if wiki_result != '']
    refined_results = []
    for d in wiki_results:
        refined_d = parse_wiki_info(d)
        refined_results.append(refined_d)
    return str(refined_results)


@tool(infer_schema=True, parse_docstring=True)
def arxiv_search(query: str) -> str:
    """Searching relevant academic papers from Arxiv for the query and returns the results.
    
    Args:
        query: The query to search for.
    """ 
    arxiv = ArxivAPIWrapper(top_k_results=5)
    arxiv_results = arxiv.run(query)
    arxiv_results = arxiv_results.split("\n\n")    
    refined_results = []
    for d in arxiv_results:
        refined_d = parse_arxiv_info(d)
        refined_results.append(refined_d)
    return str(refined_results)


# Augment the LLM with tools
tools = [add, web_search, wiki_search, arxiv_search]
tools_by_name = {tool.name: tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)
pprint(llm_with_tools.kwargs)


# Step 1: Define state
from langchain_core.messages import AnyMessage
from typing_extensions import TypedDict, Annotated
import operator

class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    llm_calls: int

# Step 2: Define model node
from langchain_core.messages import SystemMessage
def llm_call(state: dict):
    """LLM decides whether to call a tool or not"""

    systme_prompt = """You are a helpful assistant and your jobs are as below
    - performing arithmetic on a set of inputs.
    - serching relevant information from the web.
    - searching relevant academic papers from Arxiv.
    - searching relevant information from Wikipedia.
    """


    input = [SystemMessage(content=systme_prompt)] + state["messages"]
    print(f">>> State : {len(input)} / {input}")

    return {
        "messages": [llm_with_tools.invoke(input)],
        "llm_calls": state.get('llm_calls', 0) + 1
    }


# Step 3: Define tool node
from langchain_core.messages import ToolMessage

def tool_node(state: dict):
    """Performs the tool call"""
    result = []
    try:
        for tool_call in state["messages"][-1].tool_calls:
            tool = tools_by_name[tool_call["name"]]
            print(f">>> tool_call_args : {tool_call["args"]}")
            observation = tool.invoke(tool_call["args"])
            print(f">>> observation : {observation}")
            result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
        return {"messages": result}
    except Exception as e:
        print(f"Error during tool invocation: {e}")
        return {"messages": result}

# Step 4: Define logic to determine whether to end

from typing import Literal
from langgraph.graph import StateGraph, START, END

# Conditional edge function to route to the tool node or end based upon whether the LLM made a tool call
def should_continue(state: MessagesState) -> Literal["tool_node", END]:
    """Decide if we should continue the loop or stop based upon whether the LLM made a tool call"""

    messages = state["messages"]
    last_message = messages[-1]
    # If the LLM makes a tool call, then perform an action
    if last_message.tool_calls:
        return "tool_node"
    # Otherwise, we stop (reply to the user)
    return END

# Step 5: Build agent

# Build workflow
agent_builder = StateGraph(MessagesState)

# Add nodes
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("tool_node", tool_node)

# Add edges to connect nodes
agent_builder.add_edge(START, "llm_call")
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    ["tool_node", END]
)
agent_builder.add_edge("tool_node", "llm_call")

# Compile the agent
agent = agent_builder.compile()

# Invoke
from langchain_core.messages import HumanMessage
# messages = [HumanMessage(content="Add 3 and 4.")]
messages = [HumanMessage(content="한글 창제일")]
# messages = [HumanMessage(content="research of knowledge distillation")]
# messages = [HumanMessage(content="최근 한국과 미국간 관세 협상 결과")]

messages = agent.invoke({"messages": messages})
messages

{'tools': [{'function': {'description': 'Adds a and b.',
                         'name': 'add',
                         'parameters': {'properties': {'a': {'description': 'first '
                                                                            'int',
                                                             'type': 'integer'},
                                                       'b': {'description': 'second '
                                                                            'int',
                                                             'type': 'integer'}},
                                        'required': ['a', 'b'],
                                        'type': 'object'}},
            'type': 'function'},
           {'function': {'description': 'Searches the web for the query and '
                                        'returns the results.',
                         'name': 'web_search',
                         'parameters': {'properties': {'q

{'messages': [HumanMessage(content='한글 창제일', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'nghf0n3w1', 'function': {'arguments': '{"query":"한글 창제일"}', 'name': 'wiki_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 310, 'prompt_tokens': 442, 'total_tokens': 752, 'completion_time': 0.712576927, 'prompt_time': 0.021482359, 'queue_time': 0.211414078, 'total_time': 0.734059286}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cc067a02-951e-4bb3-bb0d-9a6861d3e991-0', tool_calls=[{'name': 'wiki_search', 'args': {'query': '한글 창제일'}, 'id': 'nghf0n3w1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 442, 'output_tokens': 310, 'total_tokens': 752}),
  ToolMessage(content='[{\'Page\': \'한글\', \'Summary\': "한글(문화어: 조선글)은 한국어의 공식 표기 문자로, 세종대왕이 한국어를 표기하기 위하여 1443년 창제한 \'훈민정음\'(訓民正音)을 20세기 초반부터 달리 이르는 명칭이다. \'

In [25]:
@tool(infer_schema=True, parse_docstring=True)
def think_tool(reflection: str) -> str:
    """Tool for strategic reflection on research progress and decision-making.
    
    Use this tool after each search to analyze results and plan next steps systematically.
    This creates a deliberate pause in the research workflow for quality decision-making.
    
    When to use:
    - After receiving search results: What key information did I find?
    - Before deciding next steps: Do I have enough to answer comprehensively?
    - When assessing research gaps: What specific information am I still missing?
    - Before concluding research: Can I provide a complete answer now?
    
    Reflection should address:
    1. Analysis of current findings - What concrete information have I gathered?
    2. Gap assessment - What crucial information is still missing?
    3. Quality evaluation - Do I have sufficient evidence/examples for a good answer?
    4. Strategic decision - Should I continue searching or provide my answer?
    
    Args:
        reflection: Your detailed reflection on research progress, findings, gaps, and next steps
        
    Returns:
        Confirmation that reflection was recorded for decision-making
    """
    return f"Reflection recorded: {reflection}"

# Augment the LLM with tools
tools = [think_tool]
tools_by_name = {tool.name: tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)
pprint(llm_with_tools.kwargs)

system_promt = """You are a helpful assistant"""

query = """
There a four members in my class.
CoCo likes an apple
Nana likes an banana
Momo does not like an apple
Now, We know the fruit preference of all members.
"""

result = llm_with_tools.invoke([SystemMessage(content=system_promt)] + [query])
result

{'tools': [{'function': {'description': 'Tool for strategic reflection on '
                                        'research progress and '
                                        'decision-making. Use this tool after '
                                        'each search to analyze results and '
                                        'plan next steps systematically.\n'
                                        'This creates a deliberate pause in '
                                        'the research workflow for quality '
                                        'decision-making. When to use:\n'
                                        '- After receiving search results: '
                                        'What key information did I find?\n'
                                        '- Before deciding next steps: Do I '
                                        'have enough to answer '
                                        'comprehensively?\n'
                                      

AIMessage(content="The information provided mentions three class members (CoCo, Nana, Momo) but states there are four in total. The preferences of the fourth member are not disclosed. To fully determine the fruit preferences of all four members, the preference of the unnamed fourth individual needs to be specified. \n\nWould you like to provide the fourth member's preference or clarify their name?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 313, 'prompt_tokens': 371, 'total_tokens': 684, 'completion_time': 0.717904361, 'prompt_time': 0.017700152, 'queue_time': 0.056759788, 'total_time': 0.735604513}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'finish_reason': 'stop', 'logprobs': None}, id='run--b615b98c-2ebb-4e1d-95a2-22b9717aa252-0', usage_metadata={'input_tokens': 371, 'output_tokens': 313, 'total_tokens': 684})

In [26]:
pprint(result.content)

('The information provided mentions three class members (CoCo, Nana, Momo) but '
 'states there are four in total. The preferences of the fourth member are not '
 'disclosed. To fully determine the fruit preferences of all four members, the '
 'preference of the unnamed fourth individual needs to be specified. \n'
 '\n'
 "Would you like to provide the fourth member's preference or clarify their "
 'name?')
